In [ ]:
import numpy as np
import pandas as pd
import os, shutil
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib.ticker import FuncFormatter
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import max_error
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import make_scorer
import numpy as np
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.metrics import make_scorer
import os
import glob
import plotly
from plotly import graph_objects as go
from plotly.offline import init_notebook_mode, iplot
import matplotlib.pyplot as plt
import plotly.colors as pc

## Prepare Dataset
NOTE that all the work has been done on kaggle.com, so some of steps might be skippable if You do it locally

In [ ]:
# Copying all data to working directory
if os.path.exists('/kaggle/working/clear-dataset'):
    shutil.rmtree('/kaggle/working/clear-dataset')
shutil.copytree('/kaggle/input/clear-dataset', '/kaggle/working/clear-dataset')

# Copying all data to working directory
if os.path.exists('/kaggle/working/final-nba-tester'):
    shutil.rmtree('/kaggle/working/final-nba-tester')
shutil.copytree('/kaggle/input/final-nba-tester', '/kaggle/working/final-nba-tester')

In [ ]:
# Merging tables into train and test datasets
folder_path = '/kaggle/working/clear-dataset'
train_dfs = []
test_dfs = []

for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        file_path = os.path.join(folder_path, file)
        season = file.split('-')[0]
        df = pd.read_csv(file_path)
        df['season'] = season
        if file in ['2022-2023.csv', '2023-2024.csv', '2011-2012.csv', '2003-2004.csv']:
            test_dfs.append(df)
        else:
            train_dfs.append(df)

tresholded_train_df = pd.concat(train_dfs, ignore_index=True)
tresholded_test_df = pd.concat(test_dfs, ignore_index=True)
final_test_df = pd.read_csv('/kaggle/working/final-nba-tester/2024-2025.csv')

In [ ]:
print(f'Train rows count: {len(tresholded_train_df)}')
print(f'Test rows count: {len(tresholded_test_df)}')
print(f'Final test rows count: {len(final_test_df)}')

In [ ]:
# Removing this stat as it makes no sense in this project
tresholded_train_df = tresholded_train_df.drop(columns=['gs_per_game'])
tresholded_test_df = tresholded_test_df.drop(columns=['gs_per_game'])

# Histogram of dataset (by mvp votes share)

In [ ]:
template = {
    'font_family': 'Open Sans',
    'font_size': 16,
    'font_color': 'black',
    'title_font_size': 24,
    'title_x': 0.5,
    'title_y': 1,
    'xaxis': {
        'title': 'MVP Votes Share',
        'tickfont': {'size': 16},
        'showline': True,
        'mirror': True,
        'gridcolor': '#dad9d8',
        'linecolor': '#404041',
        'showgrid': False,
        'ticklabelposition': 'outside'
    },
    'yaxis': {
        'title': 'Number of Players',
        'title_font': {'size': 20},
        'tickfont': {'size': 16},
        'showline': True,
        'mirror': True,
        'gridcolor': '#dad9d8',
        'linecolor': '#404041',
        'gridcolor': 'lightgray',
        'gridwidth': 0.5,
        'tickvals': [0, 2000, 4000, 6000, 8000, 10000],
        'ticktext': ["   0 ", "   2000 ", "   4000 ", "   6000 ", "   8000 ", "  10000 "],
        'ticklabelposition': 'outside',
        'range': [0, 10000]
    },
    'plot_bgcolor': '#f3f3f3',
    'paper_bgcolor': 'rgba(0,0,0,0)',
    'width': 600
}

colorscale = pc.sequential.Magma_r

In [ ]:
def calculate_mvp_share_distribution(df):
    if 'mvp votes share' not in df.columns:
        raise ValueError("Column 'mvp votes share' not found in the dataset!")
    
    zero_votes = (df['mvp votes share'] == 0.0).sum()
    non_zero_votes = (df['mvp votes share'] > 0.0).sum()

    return zero_votes, non_zero_votes

In [ ]:
def create_mvp_votes_distribution_plot(df, title, yaxis_range=None, yaxis_tick_interval=None, yaxis_tickvals=None, yaxis_ticktext=None):
    zero_votes, non_zero_votes = calculate_mvp_share_distribution(df)
    
    layout = go.Layout(
        **template,
        title = title,
        margin = dict(
            t=70,
            b=0,
            l=0,
            r=0
        )
    )
    
    if yaxis_range:
        layout['yaxis']['range'] = yaxis_range
    
    if yaxis_tick_interval:
        layout['yaxis']['tickvals'] = list(range(0, yaxis_range[1] + yaxis_tick_interval, yaxis_tick_interval))
        layout['yaxis']['ticktext'] = [f"{i} " for i in layout['yaxis']['tickvals']]
        layout['yaxis']['gridwidth'] = 1  # Zwiększenie szerokości linii siatki dla lepszej widoczności
    
    if yaxis_tickvals and yaxis_ticktext:
        layout['yaxis']['tickvals'] = yaxis_tickvals
        layout['yaxis']['ticktext'] = yaxis_ticktext
    
    fig = go.Figure(
        data=[
            go.Bar(
                x=["0.0", "> 0.0"],
                y=[zero_votes, non_zero_votes],
                width=0.35,
                marker=dict(
                    color=[zero_votes, non_zero_votes],
                    colorscale=colorscale,
                    showscale=True,
                    cmin=0,
                    cmax=10000,
                    colorbar=dict(
                        title=dict(
                            text='Number of<br>Players<br>&nbsp;',
                            side='top',
                        ),
                        tickvals=[0, 10000],
                        ticktext=['0', '10000'],
                        ticklen=10,
                        x=1.1,
                        tickfont=dict(size=14),
                        titlefont=dict(size=16),
                    )
                ),
                text=[zero_votes, non_zero_votes],
                textposition="outside"
            )
        ],
        layout=layout
    )
    
    return fig

fig_train = create_mvp_votes_distribution_plot(tresholded_train_df, "MVP Votes Share Distribution - Train Dataset")

fig_test = create_mvp_votes_distribution_plot(
    tresholded_test_df, 
    "MVP Votes Share Distribution - Test Dataset", 
    yaxis_range=[0, 3000], 
    yaxis_tick_interval=1000,
    yaxis_tickvals=[0, 1000, 2000, 3000],
    yaxis_ticktext=["   0 ", "   1000 ", "   2000 ", "   3000 "]
)

In [ ]:
fig_train.show(renderer='iframe')

In [ ]:
fig_test.show(renderer='iframe')

In [ ]:
# Remove all players that have been traded druing the season
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['team'] == 'TOT') & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['g'] < 50) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[g] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]


In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['pts_per_game'] < 13.0) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[pts_per_game] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['fga_per_game'] < 10.0) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[fga_per_game] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['ast_per_game'] < 2) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[ast_per_game] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['trb_per_game'] < 3) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[trb_per_game] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['fg%_total'] < 0.38) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[fg%_total] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
df_train_original = tresholded_train_df.copy()

original_row_count_train = len(tresholded_train_df)
tresholded_train_df = tresholded_train_df[~((tresholded_train_df['mp_per_game'] < 30.0) & (tresholded_train_df['mvp votes share'] == 0.0))]
filtered_row_count_train = len(tresholded_train_df)

rows_removed_train = original_row_count_train - filtered_row_count_train
print(f"[mp_per_game] Liczba usuniętych wierszy w train: {rows_removed_train}")

# Find removed rows (print them if needed)
df_train_removed = df_train_original[~df_train_original.index.isin(tresholded_train_df.index)]

In [ ]:
# Show count of rows in both datsets
print(f'Train rows count: {len(tresholded_train_df)}')
print(f'Test rows count: {len(tresholded_test_df)}')

## Figure of MVP Votes Share Distribution

In [ ]:
fig_train_after = create_mvp_votes_distribution_plot(
    tresholded_train_df,
    "MVP Votes Share Distribution - Train Dataset",
    yaxis_range=[0, 1000], 
    yaxis_tick_interval=200,
    yaxis_tickvals=[0, 200, 400, 600, 800, 1000],
    yaxis_ticktext=["  0 ", "  200 ", "  400 ", "  600 ", "  800 ", "  1000 "]
)
fig_train_after.show(renderer='iframe')

In [ ]:
tresholded_train_df_numerical = tresholded_train_df.drop(columns=['player', 'team'])
tresholded_test_df_numerical = tresholded_test_df.drop(columns=['player', 'team'])
final_test_df_numerical = final_test_df.drop(columns=['player', 'team'])

## Create train and test sets for Random Forest

In [ ]:
x_train = tresholded_train_df_numerical.drop('mvp votes share', axis=1)
y_train = tresholded_train_df_numerical['mvp votes share']

x_test = tresholded_test_df_numerical.drop('mvp votes share', axis=1)
y_test = tresholded_test_df_numerical['mvp votes share']

x_final_test = final_test_df_numerical

## Create RBO Scorer

In [ ]:
from sklearn.metrics import make_scorer
import numpy as np

def rbo_score(list1, list2, p=0.5):
"""
    Calculates the Rank-Biased Overlap (RBO) between two lists.

    Parameters:
    - list1: First ranked list
    - list2: Second ranked list
    - p: Preference parameter for top ranks (0 < p < 1)

    Returns:
    - RBO Score (float) in the range [0, 1]
"""
    # Eliminate duplicates
    list1 = list(dict.fromkeys(list1))
    list2 = list(dict.fromkeys(list2))
    depth = max(len(list1), len(list2))
    overlap = 0
    rbo_sum = 0
    for d in range(1, depth + 1):
        if d <= len(list1) and d <= len(list2) and list1[d - 1] == list2[d - 1]:
            overlap += 1
        rbo_sum += (overlap / d) * (p ** (d - 1))
    
    normalization = (1 - p) / (1 - p ** depth)
    rbo = rbo_sum * normalization
    
    return rbo

def nba_rank_scorer(y_true, y_pred):
    """
    Custom NBA Rank scorer based on RBO function
    """
    pred_df = pd.DataFrame({'true': y_true, 'pred': y_pred})
    pred_df = pd.concat([tresholded_train_df[['player', 'season']], pred_df], axis=1)

    season_rbo_scores = []

    for season in tresholded_train_df['season'].unique():
        season_df = tresholded_train_df[tresholded_train_df['season'] == season]
        true_top5 = season_df.nlargest(5, 'mvp votes share')[['player', 'mvp votes share']]
    
        pred_season_df = pred_df[pred_df['season'] == season]
        pred_top5 = pred_season_df.nlargest(5, 'pred')[['player', 'pred']]
        
        true_list = true_top5['player'].tolist()
        pred_list = pred_top5['player'].tolist()
        
        rbo = rbo_score(true_list, pred_list)
        season_rbo_scores.append(rbo)
    
    return np.mean(season_rbo_scores)

nba_rank_scorer = make_scorer(nba_rank_scorer, greater_is_better=True)

## Root Mean Squared Error scorer for comparison

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

## GridSearch with Random Forest and find the best model parameters

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('feature_selection', SelectKBest(score_func=f_regression)),
    ('regressor', RandomForestRegressor())
])

param_grid = {
    'feature_selection__k': [30],
    'regressor__n_estimators': [20, 25, 30, 45, 40, 45, 50],
    'regressor__max_samples': [0.9],
    'regressor__max_depth': [7],
    'regressor__bootstrap': [True],
    'regressor__random_state': [21]
}

model_final = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring=nba_rank_scorer,
    n_jobs=-1,
    verbose=1,
    return_train_score=True
)

########################################
start_time = time.time()
model_final.fit(x_train, y_train)
end_time = time.time()
########################################

elapsed_time = end_time - start_time
print(f"Sum of elapsed time for GridSearchCV fitting: {elapsed_time:.2f} seconds")

# Show best model parameters
print("Best parameters found: ", model_final.best_params_)

## Save model

In [ ]:
import joblib

filename = 'model.pkl'
joblib.dump(model_final.best_estimator_, filename)

print(f"Model saved to {filename}")

## Results

In [ ]:
import os
import shutil

working_dir = '/kaggle/working'
exclude_dir = 'clear-dataset'

for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    
    if item != exclude_dir and item != 'final-nba-tester':
        if os.path.isdir(item_path):
            shutil.rmtree(item_path)
        else:
            os.remove(item_path)

print("Folder '/kaggle/working' został wyczyszczony z wyjątkiem 'clear-dataset'.")

## Show results
By generating predicted top5 tables from all seasons

In [ ]:
import pandas as pd

def generate_top5_tables(tresholded_train_df, y_true, y_pred, file_prefix='top5_comparison'):
    # Create a DataFrame from predictions
    pred_df = pd.DataFrame({'true': y_true, 'pred': y_pred})
    
    # Combine with the original DataFrame to retain season information
    pred_df = pd.concat([tresholded_train_df[['player', 'season']], pred_df], axis=1)

    for season in tresholded_train_df['season'].unique():
        # Get true top 5 for the season
        season_df = tresholded_train_df[tresholded_train_df['season'] == season]
        true_top5 = season_df.nlargest(5, 'mvp votes share')[['player', 'mvp votes share']]
        
        # Get predicted top 5 for the season
        pred_season_df = pred_df[pred_df['season'] == season]
        pred_top5 = pred_season_df.nlargest(5, 'pred')[['player', 'pred']]
        
        # Merge true and predicted top 10
        top5_comparison = pd.merge(true_top5, pred_top5, on='player', how='outer', suffixes=('_true', '_pred'))
        top5_comparison['season'] = season
        
        # Save to CSV
        filename = f"{file_prefix}_{season}.csv"
        top5_comparison.to_csv(filename, index=False)
        print(f"Saved top 5 comparison for season {season} to {filename}")

# Usage
generate_top5_tables(tresholded_test_df, y_test, model_final.best_estimator_.predict(x_test))
generate_top5_tables(tresholded_train_df, y_train, model_final.best_estimator_.predict(x_train))

## Display results

In [ ]:
def display_top_5_comparison_and_rbo(file_path):
    df = pd.read_csv(file_path)

    real_top_5 = df[['player', 'mvp votes share']].sort_values(by='mvp votes share', ascending=False).head(5)
    real_top_5['mvp votes share'] = real_top_5['mvp votes share'].round(3)
    
    pred_top_5 = df[['player', 'pred']].sort_values(by='pred', ascending=False).head(5)
    pred_top_5['pred'] = pred_top_5['pred'].round(3)
    
    print("Top 5 graczy - rzeczywiste wartości (mvp votes share):")
    print(real_top_5.to_csv(index=False, sep='\t'))
    
    print("Top 5 graczy - przewidywane wartości (pred):")
    print(pred_top_5.to_csv(index=False, sep='\t'))
    
    real_players = real_top_5['player']
    pred_players = pred_top_5['player']
    rbo_value = rbo_score(real_players, pred_players)
    print(f"RBO Score: {rbo_value:.4f}")

In [ ]:
display_top_5_comparison_and_rbo('/kaggle/working/top5_comparison_2022.csv')

## Prediction on not yet ended season 2024-2025

In [ ]:
x_final_test = x_final_test[x_test.columns]
predictions = model_final.best_estimator_.predict(x_final_test)
final_test_df['prediction'] = predictions
sorted_final_test_df = final_test_df.sort_values(by='prediction', ascending=False)
sorted_final_test_df[['player', 'predictions']].head(5)

# Further there are only graphs for thesis

## Most Important Features

In [ ]:
importances = model_first.best_estimator_.named_steps['regressor'].feature_importances_

feature_importances = pd.DataFrame({
    'Feature': x_train.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 16))
sns.barplot(x='Importance', y='Feature', data=feature_importances, palette="rocket")
plt.title('Feature Importances', fontsize=16)
plt.xlabel('Importance', fontsize=14)
plt.ylabel('Feature', fontsize=14)
plt.tight_layout()

plt.savefig('/kaggle/working/feature_importances_model3.png')
plt.show()

## Most Important Features (choosen by grid search)

In [ ]:
importances = model_rmse_2.best_estimator_.named_steps['regressor'].feature_importances_

feature_importances = pd.DataFrame({
    'Feature': x_train.columns[model_rmse_2.best_estimator_.named_steps['feature_selection'].get_support()],
    'Importance': importances
}).sort_values(by='Importance', ascending=False)

plt.figure(figsize=(12, 12))
sns.barplot(x='Importance', y='Feature', data=feature_importances, palette="rocket")
plt.title('Feature Importances from Random Forest')
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.tight_layout()

plt.savefig('/kaggle/working/feature_importances_model2.png')
plt.show()

## Real vs Predicted graph

In [ ]:
import plotly.graph_objects as go
import pandas as pd

def plot_real_vs_predicted(model, x_test, y_test, template):
    y_pred_test = model.predict(x_test)
    comparison_df = pd.DataFrame({
        'Real Values': y_test,
        'Predicted Values': y_pred_test
    })

    trace_real = go.Scatter(
        x=comparison_df.index,
        y=comparison_df['Real Values'],
        mode='markers',
        name='Real value',
        marker=dict(color='rgba(0,0,0,0.6)', size=10)
    )
    trace_predicted = go.Scatter(
        x=comparison_df.index,
        y=comparison_df['Predicted Values'],
        mode='markers',
        name='Predicted value',
        marker=dict(color='rgba(239, 0, 52, 0.9)', size=10)
    )

    trace_lines = []
    for i in range(len(comparison_df)):
        trace_lines.append(
            go.Scatter(
                x=[i, i],
                y=[comparison_df['Real Values'].iloc[i], comparison_df['Predicted Values'].iloc[i]],
                mode='lines',
                line=dict(color='gray', dash='dot', width=1),
                showlegend=False
            )
        )

    fig1 = go.Figure(data=[trace_real, trace_predicted] + trace_lines)
    fig1.update_layout(
        title="Real and Predicted MVP Votes Share",
        xaxis_title="Player Index",
        yaxis_title="MVP Votes Share",
        showlegend=True,
        template='plotly',
        font=dict(family=template['font_family'], size=template['font_size'], color=template['font_color']),
        title_font_size=template['title_font_size'],
        title_x=template['title_x'],
        title_y=template['title_y'],
        xaxis=dict(
            title='Player by index',
            tickfont=dict(size=16),
            showline=template['xaxis']['showline'],
            mirror=template['xaxis']['mirror'],
            gridcolor=template['xaxis']['gridcolor'],
            linecolor=template['xaxis']['linecolor'],
            showgrid=template['xaxis']['showgrid'],
            ticklabelposition=template['xaxis']['ticklabelposition'],
            range=[-1, len(y_test)]
        ),
        yaxis=dict(
            title='MVP Votes Share',
            tickfont=dict(size=16),
            title_font=template['yaxis']['title_font'],
            showline=template['yaxis']['showline'],
            mirror=template['yaxis']['mirror'],
            gridcolor=template['yaxis']['gridcolor'],
            linecolor=template['yaxis']['linecolor'],
            gridwidth=template['yaxis']['gridwidth'],
            tickvals=[i / 10.0 for i in range(11)],
            ticktext=[f"{i / 10.0:.1f}" for i in range(11)],
            ticklabelposition=template['yaxis']['ticklabelposition'],
            range=[0.0, 1.0]
        ),
        plot_bgcolor=template['plot_bgcolor'],
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=40, r=40, t=40, b=40),
        width=1000
    )
    
    return fig1

fig1 = plot_real_vs_predicted(model_2, x_test, y_test, template)
fig1.show(renderer='iframe')


## Results Deviation graph

In [ ]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

def plot_deviation_by_cutoff_manual(template, selected_columns=['Mean Deviation', 'Max Deviation']):
    deviation_values = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
    mean_deviations = [0.11, 0.20, 0.21, 0.22, 0.24, 0.22, 0.24, 0.32, 0.28]
    max_deviations = [0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.52, 0.37]

    deviation_df = pd.DataFrame({
        'Deviation': deviation_values,
        'Mean Deviation': mean_deviations,
        'Max Deviation': max_deviations
    })
    
    deviation_df_long = pd.melt(deviation_df, id_vars='Deviation', 
                                value_vars=selected_columns,
                                var_name='Deviation Type', value_name='Deviation Value')

    fig2 = go.FigureWidget()
    for deviation_type in deviation_df_long['Deviation Type'].unique():
        filtered_df = deviation_df_long[deviation_df_long['Deviation Type'] == deviation_type]
    
        if deviation_type == 'Mean Deviation':
            colorscale = px.colors.sequential.Magma_r
        elif deviation_type == 'Max Deviation':
            colorscale = px.colors.sequential.Greys
    
        color_values = filtered_df['Deviation Value']
        cmin, cmax = 0, 1  

        fig2.add_trace(go.Bar(
            x=filtered_df['Deviation'],
            y=filtered_df['Deviation Value'],
            name=deviation_type,
            marker=dict(
                color=filtered_df['Deviation Value'],
                colorscale=colorscale,
                showscale=True,
                cmin=cmin,
                cmax=cmax,
                colorbar=dict(
                    title=f'{deviation_type.replace(" ", "<br>")}<br>&nbsp;',
                    tickvals=[0, 0.5, 1],
                    ticktext=['0', '0.5', '1'],
                    x=1.1 if deviation_type == 'Mean Deviation' else 1.25,
                    y=0.5,
                    len=0.9,
                    ticklen=10,
                    tickfont=dict(size=14),
                    titlefont=dict(size=16)
                )
            ),
            text=filtered_df['Deviation Value'],
            textposition='outside',
            textfont=dict(size=14, color='black'),
            showlegend=False
        ))

    
    fig2.update_layout(
        title="Predicted MVP Votes Share Deviation by cutoff",
        xaxis=dict(
            title='MVP Votes Share cutoff',
            tickfont=template['xaxis']['tickfont'],
            showline=template['xaxis']['showline'],
            mirror=template['xaxis']['mirror'],
            gridcolor=template['xaxis']['gridcolor'],
            linecolor=template['xaxis']['linecolor'],
            tickvals=[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
            showgrid=template['xaxis']['showgrid'],
            ticklabelposition=template['xaxis']['ticklabelposition']
        ),
        yaxis=dict(
            title='Deviation',
            titlefont=template['yaxis']['title_font'],
            showline=template['yaxis']['showline'],
            mirror=template['yaxis']['mirror'],
            gridcolor=template['yaxis']['gridcolor'],
            linecolor=template['yaxis']['linecolor'],
            gridwidth=template['yaxis']['gridwidth'],
            tickvals=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6],
            ticklabelposition=template['yaxis']['ticklabelposition'],
            range=[0, 0.6]
        ),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.0,
            xanchor="center",
            x=0.2,
            tracegroupgap=0
        ),
        font=dict(
            family=template['font_family'], 
            size=template['font_size'], 
            color=template['font_color']
        ),
        title_font_size=template['title_font_size'],
        title_x=template['title_x'],
        plot_bgcolor=template['plot_bgcolor'],
        paper_bgcolor='rgba(0,0,0,0)',
        margin=dict(l=30, r=10, t=40, b=40),
        width=1000,
    )
    
    return fig2

selected_columns = ['Mean Deviation', 'Max Deviation']
fig2 = plot_deviation_by_cutoff_manual(template, selected_columns=selected_columns)
fig2.show(renderer='iframe')

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.metrics import r2_score

# Predicting on the training and test sets
y_train_pred = model_2.predict(x_train)
y_test_pred = model_2.predict(x_test)

# Calculating R² Score
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print(f'R² on the training set: {r2_train:.4f}')
print(f'R² on the test set: {r2_test:.4f}')

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=['Training set', 'Test set'],
    horizontal_spacing=0.1
)

# Train set subplot
fig.add_trace(go.Scatter(
    x=y_train, 
    y=y_train_pred, 
    mode='markers', 
    name='Prediction',
    marker=dict(color='rgba(239, 0, 52, 0.9)', size=10)
), row=1, col=1)

# Test set subplot
fig.add_trace(go.Scatter(
    x=y_test, 
    y=y_test_pred, 
    mode='markers',
    marker=dict(color='rgba(239, 0, 52, 0.9)', size=10),
    showlegend=False,
), row=1, col=2)

fig.add_trace(go.Scatter(
    x=[min(y_train), max(y_train)], 
    y=[min(y_train), max(y_train)], 
    mode='lines', 
    name='Ideal trend',
    line=dict(color='black', dash='dash'),
    showlegend=True
), row=1, col=1)

fig.add_trace(go.Scatter(
    x=[min(y_test), max(y_test)], 
    y=[min(y_test), max(y_test)], 
    mode='lines', 
    name='Ideal', 
    line=dict(color='black', dash='dash'),
    showlegend=False 
), row=1, col=2)

for i in fig['layout']['annotations']:
    i['font'] = dict(size=20)

fig.update_layout(
    title="Real vs Predicted Values",
    title_x=template['title_x'],
    title_y=template['title_y'],
    font=dict(family=template['font_family'], size=template['font_size'], color=template['font_color']),
    title_font_size=template['title_font_size'],
    plot_bgcolor=template['plot_bgcolor'],
    paper_bgcolor=template['paper_bgcolor'],
    width=1100,
    showlegend=True,
    margin=dict(l=50, r=50, t=50, b=50)
)

fig.update_xaxes(
    title='Real MVP Votes Share',
    tickfont=dict(size=16),
    showline=template['xaxis']['showline'],
    mirror=template['xaxis']['mirror'],
    gridcolor=template['xaxis']['gridcolor'],
    linecolor=template['xaxis']['linecolor'],
    showgrid=True,
    tickvals=[i / 10 for i in range(0, 11)],
    range=[0, 1]
)

fig.update_yaxes(
    title='Predicted MVP Votes Share',
    tickfont=dict(size=16),
    title_font=template['yaxis']['title_font'],
    showline=template['yaxis']['showline'],
    mirror=template['yaxis']['mirror'],
    gridcolor=template['yaxis']['gridcolor'],
    linecolor=template['yaxis']['linecolor'],
    gridwidth=template['yaxis']['gridwidth'],
    ticklabelposition=template['yaxis']['ticklabelposition'],
    tickvals=[i / 10 for i in range(0, 11)],
    range=[0, 1]
)

fig.show(renderer='iframe')